In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Preprocess the data
### Drop irrelevent columns
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## Encode categorical variable 
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# one hot encode 'Geography

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
## combine one hot encoder columns with the original data
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## save the encoders and scalar
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
## Divide dataset into independent and dependent features
X=df.drop('Exited',axis=1)
y=df['Exited']

In [15]:
from sklearn.model_selection import train_test_split

## train test split 
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## scale these features
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [16]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [18]:
model=Sequential([
    Dense(64, activation='relu',input_shape=(x_train.shape[1],)), ## HL1 Connected with input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid') ## output layer
]

)

In [19]:
(x_train.shape[1],)

(12,)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [22]:
loss

In [23]:
## compile the model 
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [24]:
import datetime
from tensorflow.keras.callbacks import TensorBoard

## log set up tensorboard
log_dir= "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [25]:
### set up Early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [26]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 0.4454 - accuracy: 0.8091 - val_loss: 0.3912 - val_accuracy: 0.8350
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3835 - accuracy: 0.8389 - val_loss: 0.3591 - val_accuracy: 0.8525
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3577 - accuracy: 0.8534 - val_loss: 0.3523 - val_accuracy: 0.8515
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3477 - accuracy: 0.8570 - val_loss: 0.3477 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3419 - accuracy: 0.8587 - val_loss: 0.3452 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3385 - accuracy: 0.8586 - val_loss: 0.3442 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3348 - accuracy: 0.8631 - val_loss: 0.3427 - val_accuracy: 0.86

In [27]:
model.save('model.h5')

f:\Python\Churn_ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
scaler.feature_names_in_


array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [32]:
# loading the tensorboard extention 
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [29]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 10600), started 1:48:33 ago. (Use '!kill 10600' to kill it.)

In [30]:
!kill 10600


kill: 10600: No such process


In [31]:
!kill 10600

kill: 10600: No such process


In [33]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 10600), started 1:51:33 ago. (Use '!kill 10600' to kill it.)

In [34]:
!kill 10600


kill: 10600: No such process


In [35]:
%tensorboard --logdir logs/fit --port=6007


In [36]:
%tensorboard --logdir logs/fit --port=6007

Reusing TensorBoard on port 6007 (pid 4936), started 0:04:59 ago. (Use '!kill 4936' to kill it.)

In [37]:
!kill 4936

kill: 4936: No such process
